In [ ]:
import random 
import numpy as np 

# Basic parameters of a 117M GPT2 network 
gpt2_small = {
    "n_head": 12,
    "encoder_path": "gs://openwebtext/stuff/encoder",
    "n_vocab": 50257,
    "embed_dropout": 0.1,
    "lr": 0.00025,
    "warmup_steps": 2000,
    "beta1": 0.9,
    "beta2": 0.98,
    "epsilon": 1e-9,
    "opt_name": "adam",
    "weight_decay": 0.01,
    "train_batch_size": 32,
    "attn_dropout": 0.1,
    "train_steps": 10000,
    "eval_steps": 10,
    "max_steps": 500000,
    "data_path": "gs://connors-datasets/openwebtext/",
    "scale": 0.2886751345948129,
    "res_dropout": 0.1,
    "predict_batch_size": 1,
    "eval_batch_size": 32,
    "iterations": 500,
    "n_embd": 768,
    "input": "openwebtext",
    "model": "GPT2",
    "model_path": "gs://connors-models/GPT2-117M",
    "n_ctx": 1024,
    "predict_path": "logs/predictions.txt",
    "n_layer": 12
}


# Running parameters, each experiment needs a unique name, whether the TPU should be preemptible
# what type of TPU to use (GPUs TODO) and the actual model parameters
experiment_base = {
    "name": "gpt2_small",
    "preemptible": True,
    "accelerator_type": "v2-8",
    "model_params": gpt2_small,
}

In [ ]:
# A class defining a hyperparameter to vary 
class HyperParameter(object):
    def __init__(self, name, distribution, model=True, dtype=None, values=None):
        self.name = name # Name of the parameter to vary 
        # Nên sử dụng phân phối nào để tạo ra các giá trị,
        # một trong các giá trị lưới, mẫu, đồng nhất hoặc hình học
        self.distribution = distribution 
        self.values = values # các giá trị được sử dụng theo sự phân phối 
        self.dtype = dtype  # Liệu có float hay int hay không nếu cần 
        self.model = model # tham số thuộc model parameter hay không 

        # kiểm tra điều kiện xem phân phối hiện tại có phải grod hay là sample không 
        # Phân phối Grid tạo ra một tập hợp các giá trị bằng cách chia đều khoảng giá trị cho trước thành các phần bằng nhau.
        # Phân phối Sample tạo ra một tập hợp các giá trị ngẫu nhiên từ một khoảng giá trị cho trước.
        if distribution == "grid" or distribution == "sample":
            # kiểm tra xem kiểu values có phải list 
            assert type(values) == type([]) # nếu không thì dừng trương trình 
            # gán giá trị cho values 
            self.values = values 
            self.index = 0

        # trường hợp nếu phân phối hiện tại là uniform 
        # một công cụ hữu ích để điều chỉnh các siêu tham số có thể có nhiều giá trị khác nhau
        elif distribution == "uniform":
            # kiểm tra xem kiểu values có phải list 
            assert type(values) == type([])
            # và độ dài của values == 2 
            assert len(values) == 2 
        
        # trường hợp phân phối hiện tại là geomitric 
        # là phân phối để tạo ra một chuỗi các giá trị giảm dần theo câps số nhân 
        # Thường được sử dụng để điều chỉnh các  tham số liên quan đến  tốc độ học tập (learning rate) 
        # hoặc số lượng epoch trong quá trình huấn luyện mô hình.
        elif distribution == "geometric":
            # kiểm tra xem kiểu values có phải list 
            assert type(values) == type([])
            # và độ dài của values == 2 
            assert len(values) == 2 

    # Thiết lập phương thức get_value được sử dụng để lấy giá trị tiếp theo 
    # của 1 siêu tham số được biểu diễn bởi lớp Hyperparameter
    def get_value(self):
        # Đơn giản chỉ cần lặp lại 1 danh sách của các giá trị 
        # kiểm tra xem phân phối có phải alf grid 
        if self.distribution == "grid":
            # kiểm tra xem chỉ số hiện tại có nhỏ hơn độ dài dnah scahs các giá trị 
            if self.index < len(self.values):
                # nếu có lấy giá trị tại vị trí hiện tại và gán nó cho val 
                val = self.values[self.index]
                # Tăng chỉ số index lên 1 
                self.index += 1
                # sau đó trả về giá trị val 
                return val 
            # trwuongf hợp nếu = hoặc lớn hơn thì đưa da 1 cảnh báo lỗi 
            else:
                raise RuntimeError("{} ran out of values!".format(self.name))

        # Trường hợp phân phối lấy mẫu ngẫu nhiên từ 1 danh sách các giá trị 
        elif self.distribution == "sample":
            # trả về 1 giá trị values ngẫu nhiên 
            return random.sample(self.values)
        
        # Mẫu từ phân phối đồng đều 
        elif self.distribution == "uniform":
            # kiểm tra xem kiểu dtype  = float 
            if self.dtype == "float":
                # trả về ngẫu nhiên 1 giá trị ngẫu nhiên từ giá 
                # trị tối thiểu đến tối đa trong list values 
                return random.uniform(self.values[0], self.values[1])
            # trường hợp khác 
            else :
                # trả về một giá trị nguyên phép lấy tương tự 
                return int(random.uniform(self.values[0], self.values[1]))
        
        # Mẫu từ phân phối "hình học"
        # Một mẫu được rút ra từ phân bố đồng đều từ log(A) đến log(B) 
        # và sau đó lũy thừa để tạo ra giá trị
        elif self.distribution == "geometric":
            # kiểm tra kiểu của dtype 
            if self.dtype == "float":
                # nếu là float trả về tham số e mũ một giá trị tham số ngẫu nhiên 
                return np.exp(np.random.uniform(np.log(self.values[0]), np.log(self.values[1])))
            else:
                # tương tự như trên nhưng ở đây kết quả là một giá trị nguyên int 
                return int(np.exp(np.random.uniform(np.log(self.values[0]), np.log(self.values[1]))))



In [ ]:
#  Đưa ra các tham số cơ bản của mô hình, danh sách các tham số sẽ thay đổi và một số, tạo ra số lượng thử nghiệm để chạy
# Given the base parameters of the model, list of parameter to vary and a number, generates number amount of experiments to run
def generate_experiments(base, parameters, number):
    # Tạo một danh sách trống để lưu trữ các cấu hình thử nghiệm 
    experiments = []
    # lặp qua number lần 
    for i in range (number):
        # copy dữ liệu đầu vào cấu hình cơ sở base gán cho biến ex 
        ex = base.copy()
        # Cập nhật tên và thư mục mô hình 
        # Thêm chuỗi "-" + str(1) vào tên và thư mục mô hình của cấu hình ex để phân 
        # biệt các thử nghiệm khác nhau 
        ex["name"] = ex["name"] + "-" + str(i)
        # Sử dụng cú pháp ex["model_params"]["model_dir"] để truy cập giá trị của khóa "model_dir" 
        # trong từ điển "model_params" của cấu hình ex. Điều này tạo ra đường dân duy nhất cho mỗi mô 
        # hình được sử dụng trong thí nghiệm 
        ex["model_params"]["model_dir"] = ex["model_params"]["model_dir"] + "-" + str(i)

        # gán giá trị cho các thm số 
        # Lặp qua mỗi phần tử trong danh scahs parameters danh sách này chứa các đối tượng 
        # có thuộc tính model và name và phương thức get_value 
        for p in parameters:
            # kiểm tra xem tham số có thuộc về tham số của mô hình 
            if p.model :
                # ta gán giá trị siêu tham số cho khóa "name"  trong từ điển "model_params" của câú hình ex
                ex["model_params"][p.name] = p.get_value()
            # truuwongf hợp không là tham số của model 
            else: 
                # gán trực tiếp giá trị cho siêu tham số đó trong từ điển ex. 
                ex[p.name] = p.get_value()

        # Thêm cấu hình từ điển ex đã được cập nhật vào danh sách lưu trữ câú hình thử nghiệm 
        experiments.append(ex)

    # trả về danh sách các cấu hình thửu nghiệm đã được đào tạo 
    return experiments 



# Cấu hình các bộ siêu tham số 
parameters = [
    # phân bổ hình học  tham số lr  phân phối để tạo ra một chuỗi các giá trị giảm dần theo câps số nhân 
    HyperParameter("lr", "geometric", values=[1e-5, 1e-1], dtype="float"),
    # phân phối mẫu điều chỉnh các siêu tham số có thể có nhiều giá trị khác nhau
    HyperParameter("input", "sample", values=["openwebtext", "openwebtext_long", "openwebtext_longbiased"]),
    # phân phối đồng bộ 
    HyperParameter("n_layers", "uniform", values=[12, 24])
]

# This is what is exported to Overrunner to run
experiments = generate_experiments(experiment_base, parameters, 10)